# Agents to Research and Write an Article

In [ ]:
%pip install crewai crewai_tools langchain_community

In [1]:
import warnings
warnings.filterwarnings("ignore")

- Import from the crewAI libray.

In [2]:
from crewai import Agent, Task, Crew, LLM

- LLM for the agent

In [3]:
import os
from utils import get_gemini_api_key
from langchain_community.llms import HuggingFaceHub, Together

llm = LLM(
    model="gemini/gemini-1.5-pro-latest",
    api_key=get_gemini_api_key(),
    temperature=0.2,
)

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

In [4]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True,
    llm=llm,
)

### Agent: Writer

In [5]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True,
    llm=llm
)

### Agent: Editor

In [6]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True,
    llm=llm
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [7]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [8]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [9]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [10]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=True
)

## Running the Crew

In [11]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: d5867f2b-a239-4c48-8396-feeb2cf66cff                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 6b551fbc-672d-4442-b539-636d94ec57e2
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 6b551fbc-672d-4442-b539-636d94ec57e2
       Status: Executing Task...
    └── 🤖 Agent: Content Planner
            Status: In Progress

# Agent: Content Planner
## Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


🚀 Crew: crew
└── 📋 Task: 6b551fbc-672d-4442-b539-636d94ec57e2
       Status: Executing Task...
    └── 🤖 Agent: Content Planner
            Status: In Progress
        └── 🧠 Thinking...

...



# Agent: Editor
## Final Answer: 
# Demystifying Artificial Intelligence: A Practical Guide for Everyone

Have you ever wondered how Netflix suggests movies you might like or how your phone understands your voice commands?  These are examples of Artificial Intelligence (AI) at work. AI is rapidly transforming our world, and while it may seem complex, its applications are already integrated into our daily lives. This article provides a clear and concise overview of AI, its current uses, and its potential impact.

## What is Artificial Intelligence?

Artificial intelligence is the ability of a computer or machine to mimic human intelligence. This includes learning from experience, adapting to new information, and performing tasks that typically require human cognition, such as problem-solving and decision-making. AI encompasses various subfields, including machine learning and deep learning. Machine learning algorithms enable computers to learn from data without explicit programming. D

🚀 Crew: crew
├── 📋 Task: 6b551fbc-672d-4442-b539-636d94ec57e2
│      Assigned to: Content Planner
│      Status: ✅ Completed
│   └── 🤖 Agent: Content Planner
│           Status: ✅ Completed
├── 📋 Task: 329e6985-210e-4c20-a2a9-b485cca4779b
│      Assigned to: Content Writer
│      Status: ✅ Completed
│   └── 🤖 Agent: Content Writer
│           Status: ✅ Completed
└── 📋 Task: 03d3d804-4515-44f7-b87a-dcc5f9994bbb
       Status: Executing Task...
    └── 🤖 Agent: Editor
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: 6b551fbc-672d-4442-b539-636d94ec57e2
│      Assigned to: Content Planner
│      Status: ✅ Completed
│   └── 🤖 Agent: Content Planner
│           Status: ✅ Completed
├── 📋 Task: 329e6985-210e-4c20-a2a9-b485cca4779b
│      Assigned to: Content Writer
│      Status: ✅ Completed
│   └── 🤖 Agent: Content Writer
│           Status: ✅ Completed
└── 📋 Task: 03d3d804-4515-44f7-b87a-dcc5f9994bbb
       Assigned to: Editor
       Status: ✅ Completed
    └── 🤖 Agent: Editor
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 03d3d804-4515-44f7-b87a-dcc5f9994bbb                                                                     │
│  Agent: Editor                                                                                                  │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: d5867f2b-a239-4c48-8396-feeb2cf66cff                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

- Display the results of your execution as markdown in the notebook.

In [13]:
from IPython.display import Markdown
Markdown(result.raw)

# Demystifying Artificial Intelligence: A Practical Guide for Everyone

Have you ever wondered how Netflix suggests movies you might like or how your phone understands your voice commands?  These are examples of Artificial Intelligence (AI) at work. AI is rapidly transforming our world, and while it may seem complex, its applications are already integrated into our daily lives. This article provides a clear and concise overview of AI, its current uses, and its potential impact.

## What is Artificial Intelligence?

Artificial intelligence is the ability of a computer or machine to mimic human intelligence. This includes learning from experience, adapting to new information, and performing tasks that typically require human cognition, such as problem-solving and decision-making. AI encompasses various subfields, including machine learning and deep learning. Machine learning algorithms enable computers to learn from data without explicit programming. Deep learning, a subset of machine learning, uses artificial neural networks with multiple layers to analyze complex data.  For example, spam filters use machine learning, while self-driving cars rely on deep learning.

AI systems are designed to perform tasks that usually require human intelligence. These systems learn from experience, adapt to new information, and make decisions based on training data.  Different types of AI exist, including narrow or weak AI, designed for specific tasks like playing chess or recommending products. General AI, possessing human-level intelligence, remains a theoretical concept.  It's important to understand that AI systems are only as good as their training data, and they can reflect existing societal biases if the data is biased.  Further development and research are ongoing to address these challenges.

## How AI is Changing the World

AI significantly impacts various sectors. In healthcare, it assists with diagnosis, drug discovery, and personalized medicine.  Financial institutions use AI for fraud detection, algorithmic trading, and risk assessment.  Self-driving cars, traffic optimization systems, and autonomous drones are transforming transportation.  Entertainment platforms utilize AI for content recommendations and personalized experiences, with AI-generated art and music emerging as new creative fields. Businesses employ AI for customer service chatbots, marketing automation, and predictive analytics.

These applications are just the beginning. AI has the potential to revolutionize industries by optimizing supply chains and developing new materials, driving innovation and efficiency. AI-powered virtual assistants help us manage schedules and answer questions, while personalized recommendations enhance online shopping. In the workplace, AI automates repetitive tasks, allowing human workers to focus on more creative and strategic endeavors. While job displacement is a valid concern, AI also creates new opportunities in areas like AI development, data science, and AI ethics. Adapting to these changes and acquiring relevant skills is crucial for thriving in an AI-driven world.


## The Future of AI

The future of AI is filled with possibilities. Emerging trends include explainable AI (making AI decision-making more transparent), edge computing (bringing AI processing closer to the data source), and quantum computing (potentially increasing AI capabilities exponentially). These advancements could lead to breakthroughs in personalized medicine, climate modeling, and materials science.  However, addressing potential societal impacts like job displacement and ethical considerations surrounding bias and privacy is crucial.  As AI becomes more integrated into our lives, developing ethical guidelines and regulations to ensure responsible development and deployment is essential.

Continued research and development are crucial for navigating the future of AI.  Experts are actively working on creating ethical frameworks and guidelines for AI development and deployment.  The ongoing dialogue involves researchers, policymakers, and the public to ensure AI benefits all of humanity.

## Getting Started with AI

Numerous resources are available for those interested in learning more about AI. Online courses, books, and podcasts offer accessible introductions to AI concepts and applications.  Exploring AI tools in daily life or business can provide practical experience.  Engaging with the AI community through online forums and conferences can help individuals stay updated on the latest developments.

Whether you're a student, professional, or simply curious about technology, there are many ways to explore AI.  Start with areas of interest.  For example, explore AI applications in medical diagnosis if you're interested in healthcare.  Business owners can consider how AI can improve customer service or marketing.  The key is to be curious, explore, and engage with the evolving world of AI.

## Conclusion

AI is transforming our world. Understanding its applications and potential impact is essential for everyone. From healthcare to entertainment, AI is reshaping industries and creating new opportunities.  While the future of AI presents both promise and challenges, informed discussions and responsible AI development are key to harnessing its power for a better future.

What are your thoughts on the future of AI? Share your predictions and concerns in the comments below!  We encourage you to explore the provided resources to learn more about this fascinating field. (This section would include links to curated resources).